<a href="https://colab.research.google.com/github/jasleenkaursandhu/Reproducing-chest-xray-report-generation-boag/blob/main/knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# KNN Model for Feature Extraction and Nearest Neighbor Analysis
# This notebook implements a KNN-based approach for chest X-ray report generation.
# It extracts DenseNet features from images and finds the most similar training images for each test image.

In [15]:
# Import necessary libraries
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tqdm
from collections import defaultdict
import pickle
!pip install pydicom
import pydicom  # Note: Original used 'dicom' but we'll use 'pydicom' which is the current version
from time import gmtime, strftime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.9 MB/s eta 0:00:00


In [16]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/mimic-cxr-project'
!mkdir -p {base_path}/data
!mkdir -p {base_path}/output

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
# Import the report parser module
import sys
sys.path.append(f"{base_path}/modules")
from report_parser import parse_report, MIMIC_RE
print("Successfully imported report parser module")

Successfully imported report parser module


In [18]:
# Load train and test data
data_dir = os.path.join(base_path, 'data')
files_path = os.path.join(base_path, 'files')
output_dir = os.path.join(base_path, 'output')
features_dir = os.path.join(base_path, 'features')

train_df = pd.read_csv(os.path.join(data_dir, 'train.tsv'), sep='\t')
test_df = pd.read_csv(os.path.join(data_dir, 'test.tsv'), sep='\t')

print(f"Train data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")

Train data shape: (824, 3)
Test data shape: (382, 3)


In [19]:
# Map each dicom to its report identifier (study_id in our case)
# Note: Original code used 'rad_id', we'll use 'study_id' based on our data structure
report_id_column = 'study_id'
if report_id_column in train_df.columns:
    rad_lookup = dict(train_df[['dicom_id', report_id_column]].values)
    print(f"Created lookup using {report_id_column}")
else:
    print(f"Warning: {report_id_column} not found in columns: {train_df.columns.tolist()}")
    rad_lookup = {}

print("Sample of lookup dictionary:")
print(dict(list(rad_lookup.items())[:5]))

Created lookup using study_id
Sample of lookup dictionary:
{'dfd9a06c-2994892e-f4a6bc1c-f6ec4803-283e5005': 55606773, 'ab50de59-ebf1d4df-20709b76-e8df1c5b-02561a38': 55244750, 'fafaee95-e11d24c5-ad39cfcd-302ac853-4a6f16ac': 58401243, 'fd53f5b0-0070f205-6c47cbe6-eaf7140d-12b09066': 57100718, '78e28f8c-fc928714-2cdc13f2-e6e45d40-89cb7eca': 53346921}


In [20]:
# Path to neighbors file
neighbors_path = os.path.join(base_path, 'output', 'top100_neighbors.pkl')

# Check if neighbors file exists
if os.path.exists(neighbors_path):
    print(f"Loading neighbors from {neighbors_path}")
    with open(neighbors_path, 'rb') as f:
        neighbors = pickle.load(f)
    print(f"Loaded neighbors for {len(neighbors)} test images")
else:
    print(f"Neighbors file not found at {neighbors_path}")
    print("Please run the feature extraction and nearest neighbor search first.")

Loading neighbors from /content/drive/MyDrive/mimic-cxr-project/output/top100_neighbors.pkl
Loaded neighbors for 380 test images


In [21]:
# Define the path to the reports directory
files_path = os.path.join(base_path, 'files')

# Generate reports for each test image
generated_reports = {}

for pred_dicom in tqdm.tqdm(test_df.dicom_id):
    # Skip if we don't have neighbors for this test image
    if pred_dicom not in neighbors:
        print(f"Warning: No neighbors for {pred_dicom}")
        continue

    nn = neighbors[pred_dicom]

    found = False
    i = 0  # Start with the closest neighbor

    while not found and i < len(nn):
        nearest_dicom = nn[i]

        # Skip if we don't have a report ID for this training image
        if nearest_dicom not in rad_lookup:
            i += 1
            continue

        nearest_train_report_id = rad_lookup[nearest_dicom]

        # Get corresponding subject_id for the training image
        subject_row = train_df[train_df.dicom_id == nearest_dicom]
        if len(subject_row) == 0:
            i += 1
            continue

        subject_id = subject_row.iloc[0]['subject_id']

        # Construct path to the report
        subject_prefix = f"p{str(subject_id)[:2]}"
        subject_dir = f"p{subject_id}"
        study_dir = f"s{nearest_train_report_id}"
        report_file = f"{study_dir}.txt"
        report_path = os.path.join(files_path, subject_prefix, subject_dir, report_file)

        # Parse the report
        try:
            if os.path.exists(report_path):
                report = parse_report(report_path)

                # If the report has a findings section, use it
                if 'findings' in report:
                    found = True
                    generated_reports[pred_dicom] = report['findings']
        except Exception as e:
            pass

        i += 1

    if not found:
        print(f"Warning: Could not find a valid report for {pred_dicom}")

print(f"Generated reports for {len(generated_reports)}/{len(test_df)} test images")

 11%|█▏        | 43/382 [00:12<01:42,  3.30it/s]

 89%|████████▉ | 341/382 [01:02<00:04,  9.90it/s]

100%|██████████| 382/382 [01:06<00:00,  5.73it/s]

Generated reports for 380/382 test images


In [22]:
# Save the generated reports to a TSV file
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

pred_dir = os.path.join(base_path, 'output')
os.makedirs(pred_dir, exist_ok=True)

pred_file = os.path.join(pred_dir, 'knn.tsv')
print(f"Saving predictions to {pred_file}")

with open(pred_file, 'w') as f:
    print('dicom_id\tgenerated', file=f)
    for dicom_id, generated in sorted(generated_reports.items()):
        # Clean up the text (remove any tabs)
        cleaned_text = generated.replace('\t', ' ')
        print(f'{dicom_id}\t{cleaned_text}', file=f)

print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

2025-04-13 21:18:36
Saving predictions to /content/drive/MyDrive/mimic-cxr-project/output/knn.tsv
2025-04-13 21:18:37


In [23]:
# Display sample of generated reports
sample_count = min(3, len(generated_reports))
sample_dicoms = list(generated_reports.keys())[:sample_count]

for dicom_id in sample_dicoms:
    print(f"\nSample report for {dicom_id}:")
    report_text = generated_reports[dicom_id]

    # Print preview of the report (first 200 characters)
    if len(report_text) > 200:
        print(report_text[:200] + "...")
    else:
        print(report_text)


Sample report for fea90b5d-a059ecc8-b5e68f8d-b7f33ed9-1d32d429:
heart size upper limits are normal. normal pulmonary vascularity. lungs are clear. no pleural effusions. no pneumothorax.

Sample report for 06ba097a-bf8b917c-851d0e95-886936a1-90964781:
no definite focal consolidation is seen. there is no large pleural effusion or pneumothorax. the cardiac mediastinal silhouettes are grossly stable.

Sample report for 648449fa-5e173b2a-87663d57-4a0fbfc0-138e42c7:
there is no definite focal consolidation, pleural effusion, or pneumothorax. compared to the prior study, there is mild prominence of vascular markings likely reflecting mild pulmonary edema. opacitie...
